In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = [12,8]

## Training (Naive) Bayes

As we have shown in the previous lectures training a Bayes classifier amounts to finding the conditional probability distributions of the features $\mathbf{f}$ given the class label $c$. 

$$P(\mathbf{F}=\mathbf{f}|C=c)$$

In case of the Naive bayes classifier the $n_f$ features are conditionally independent 

$$P(\mathbf{F}=\mathbf{f}|C=c)=\prod_{i=0}^{n_f-1} P(F_i=f_i|C=c)$$

so we can estimate   probality distribution  for each feature separately. 

### Categorical features

In case of categorical features each $x_i$ has a finite $m_i$  number of possible values (categories) that, without any loss of generality we can  assume, take values $0,\ldots,m_i-1$. In the same way we  will assume that the class labels take $n_c$ integer values $c=0,\ldots,n_c-1$. 

So for each feature we have  to estimate  $n_c\times m_i$ probabilities 

$$p^{(c)}_{ij} = P(F_i = j|C = c)$$

Of course they are not all idependent. Normalisation requires

$$\sum_{j=0}^{m_i-1} p^{(c)}_{ij}=1,\quad i=0,\ldots,n_f,\; c=0,\ldots,n_c$$

Let $\mathbf{X}$ denote the $n_s \times n_f$ matrix of training data where $n_s$ is the number of samples. So $X_{hi}$ denotes the value of the ith   feature in  sample $h$. Let  $y_h$ denote the corresponding label

$$X_{hi} \in \{0,\ldots,m_i-1\},\quad  y_h \in \{0,\ldots,n_c-1\},\quad h = 0,\ldots,n_s-1$$

Let's introduce some more notation. $n_c$ will denote the number of samples belonging to class $c$

$$n^{c}=\sum_{h=0}^{n_s-1}\delta_{y_h,c},\qquad \delta_{a,b}=
\begin{cases}
1 & a=b\\
0 & a\neq b
\end{cases}$$

and $n^{(c)}_{ij}$ the number of samples of class $c$ with feature $i$ equal to  $j$

$$n^{(c)}_{ij} = \displaystyle
\sum_{h=0}^{n_s-1} \delta_{X_{hi},j}\delta_{y_h,c}
$$

To estimate the probabilities we will use a smoothed estimator 

$$p^{(c)}_{ij} = \frac{n^{(c)}_{ij}+\alpha}{n^c+m_i\alpha} $$

where $\alpha\ge 0$ is a smoothing parameter. The use of  non-zero smoothing parameter ensures a non-vanishing probability even when $n^{(c)}_{ij}=0$. 

## Example: Car evaluation data set

As an example we will use the [car evaluation dataset](http://archive.ics.uci.edu/ml/datasets/Car+Evaluation) from [UCI Machine Learning repository](http://archive.ics.uci.edu/ml/). It contains 1728 samples with six atttributes (features) each. The class label is the evaluation of the car: $\{unacc, acc, good, vgood\}$.

All six parameters are categorical and the data contains exactly one sample for each possible combination of attributes values (in this respect this is quite peculiar dataset).

As before we will use pandas to read and proccess the data.

In [ ]:
cars_data = pd.read_csv("../../Data/Cars/car_data.csv", names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'])

In [ ]:
cars_data.head()

In [ ]:
cars_data.info()

Method `groupby`  divides the data frame into goups based on the value of the given colum(s)

In [ ]:
cars_by_class = cars_data.groupby('class')

The size of each group can be calculated using method `size`

In [ ]:
cars_by_class.size()

As we can see the classes are not very well ballanced with relatively small number of cars in two best  classes.  So I have decided to join those two classes together introducing a new classification

In [ ]:
def bargain(c):
    if c in ['good', 'vgood']:
        return 'good'
    elif c=='acc':
        return 'fair'
    else:
        return 'bad'

In [ ]:
cars_data['bargain'] = cars_data['class'].apply(bargain)
cars_data = cars_data.drop('class',axis=1)

We start by dividing the data set into training and testing. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
seed = 678565

In [ ]:
cars_train, cars_test = train_test_split(cars_data, train_size=0.75, random_state=seed)

In [ ]:
cars_train['bargain'].value_counts()

In [ ]:
cars_test['bargain'].value_counts()

Function `train_test_split` has an option to _stratify_ data based on the values of one colum

In [ ]:
cars_train, cars_test = train_test_split(cars_data, train_size=0.75, stratify=cars_data['bargain'],
                                         random_state = seed)

In this way the split was done separately for each  class label. That way we obtain as a result slightly more balanced sets

In [ ]:
cars_train['bargain'].value_counts()

In [ ]:
cars_test['bargain'].value_counts()

We will use the Naive Bayes.
There are many ways that we can calculate the estimators. We can start by grouping the dataframe according to  feature values

In [ ]:
cars_training_grouped = cars_train.groupby(['bargain', 
                                            'buying',
                                            'maint',
                                            'doors', 
                                            'persons', 
                                            'lug_boot',
                                            'safety'])

and count the size of each group

In [ ]:
group_counts=cars_training_grouped.size()

In [ ]:
group_counts

The `sum` method can make a partial sums which we can use to extract $n^{c}_{ij}$ values

In [ ]:
# The level argument list the levels not summed over i.e. left in the result.
group_counts.groupby(level = ['bargain', 'buying'] ).sum()

and finally calculate the probabilities

In [ ]:
(group_counts.groupby(level = ['bargain', 'buying'] ).sum() +1 )/(group_counts.groupby(level='bargain').sum()+4)

## Scikit-learn

However if you look closely you will notice one problem: the 'good' class does not contain any values for 'high' and 'vhigh'  attribute values! That means of course that they are zero, but it complicates the calculations. Instead of fixing this by hand we will use the tools from the [scikit-learn](https://scikit-learn.org/stable/index.html) library. This library has a class implementing just what we need

In [ ]:
from sklearn.naive_bayes import CategoricalNB

In [ ]:
cnb = CategoricalNB(alpha=1)

However this classifier requires the  class labels and attributes to be  integer numbers counted from zero. Fortunatelly scikit-learn also includes a class for converting from labels to ordinals:

In [ ]:
from sklearn.preprocessing import  OrdinalEncoder

In [ ]:
features_encoder = OrdinalEncoder(dtype='int32')
train_encoded = features_encoder.fit_transform(cars_train)

In [ ]:
features_encoder.feature_names_in_

In [ ]:
np.bincount(train_encoded[:,-1])

In [ ]:
cnb.fit(train_encoded[:,:-1], train_encoded[:,-1])

We can view the learned probabilities of this classifier  using its `feature_log_prob` attribute:

In [ ]:
np.exp( cnb.feature_log_prob_[0] )

Comparing with our calculations we see a almost perfect agreement. Mowever we get all four probabilities in the last line like required. The two values are not zero because of smoothing. 

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(train_encoded[:,-1], cnb.predict(train_encoded[:,0:-1]), normalize=True)

After training the classifier we can use it to  make predictions on the test set

In [ ]:
test_encoded = features_encoder.transform(cars_test)
encoded_test_class = test_encoded[:,-1]

In [ ]:
predicted_test_class = cnb.predict(test_encoded[:,:-1])

and test them

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(encoded_test_class, predicted_test_class, normalize=True)

Actually the classifier has a method for predicting and measuring accuracy

In [ ]:
cnb.score(test_encoded[:,:-1], encoded_test_class)

which unsuprisingly gives same results

As a last check I will look at the class distribution in the predicted and real labels. As it may happen that with unbalanced  classes  one class can be e.g. totaly misclassified without affecting accuracy. 

In [ ]:
np.bincount(predicted_test_class.astype('int64'))

In [ ]:
np.bincount(encoded_test_class.astype('int64'))

## Multiclass metrics

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay

### Confusion matrix

The accuracy  generalises intuitively to non-binary classification.  The confusion matrix also generalises to this case.  

In [ ]:
pred = predicted_test_class
true = encoded_test_class

In [ ]:
confusion_matrix(true, pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(true, pred, normalize='true', 
                                        display_labels=features_encoder.categories_[-1]);

### Averaging 

Other metrics are defined only for binary classification problem as they rely on a 2x2 confusion matrix.  A way to generalise them to multiclass classification it to treat a $k$ class classifcation problem as $k$ binary classification problems: class   $C_i$ against the rest. We combine the final score out of binary metrics for each binary classification. 

<table style="font-size:1.5em;">
<tr> <th>         </th> <th colspan=2>predicted</th></tr>
    <tr> <th>actual</th> <th>N  </th> <th>P  </th> <th>total</th></tr>
    <tr> <th> N       </th> <td>TN </td> <td>FP </td> <td>N</td></tr>
    <tr> <th> P       </th> <td>FN </td> <td>TP </td> <td>P</td></tr>
</table>

In [ ]:
def stat(y_true, y_pred, c):
    lbl_true = np.where(y_true==c,1,0)
    lbl_pred = np.where(y_pred==c,1,0)
    (TN,FP),(FN,TP) = confusion_matrix(lbl_true, lbl_pred)    
    return TP, FP,FN, TN

In [ ]:
for i in range(3):
    print( stat(true,pred, i))

Once we have the statistics for each binary classifier we can combine the together. We will consider _micro_ and _macro_ averaging. 

In [ ]:
mcf = multilabel_confusion_matrix(true, pred)
print(mcf)

### Micro averaging

In micro averaging we first calculate the summary values of TP, FP, TN and TN and use them to calculate the total score. 
We will start with _recall_ which is just another name for true positives rate.

$$Recall_\mu = \frac{\sum_i TP_i}{\sum_i(TP_i+FN_i)}$$

In [ ]:
num = 0
den = 0
for i in range(3):
    ((tn, fp), (fn, tp)) =  mcf[i]
    num += tp
    den += tp+fn
recall_mu = num/den    
print(num, den, recall_mu   )

$$Precision_\mu = \frac{\sum_i TP_i}{\sum_i(TP_i+FP_i)}$$

In [ ]:
num = 0
den = 0
for i in range(3):
    ((tn, fp), (fn, tp)) =  mcf[i]
    num += tp
    den += tp+fp
precision_mu = num/den    
print(num, den, precision_mu    )

and $F_1$ is then  harmonic mean of the two

$$F_\mu = 2\cdot\frac{Precision_\mu\cdot Recall_\mu}{Precision_\mu + Recall_\mu}$$

In [ ]:
print(2*precision_mu*recall_mu/(precision_mu+recall_mu))

#### Problem

  Show that $Recall_\mu$ = $Precision_\mu = Accuracy$

There is no surprise that scikit-learn library has functions to calculate those metrics

In [ ]:
from sklearn.metrics import *

In [ ]:
print(
    recall_score(encoded_test_class, predicted_test_class, average='micro'),
    precision_score(encoded_test_class, predicted_test_class, average='micro'),
    f1_score(encoded_test_class, predicted_test_class, average='micro'))

### Macro averaging

With macro averaging we  calculate  score for each binary classifier separately and average them. So for recall

$$Recall_M = \frac{1}{k}\sum_{i=0}^{k-1}\frac{TP_i}{TP_i+FN_i}$$

In [ ]:
tot = 0
for i in range(3):
    ((tn, fp), (fn, tp)) =  mcf[i]
    tot +=tp/(tp+fn)
rec = tot/3    
print(rec)

In [ ]:
recall_score(encoded_test_class, predicted_test_class, average='macro')

and for precision

$$Precision_M = \frac{1}{k}\sum_{i=0}^{k-1}\frac{TP_i}{TP_i+FP_i}$$

In [ ]:
tot = 0
for i in range(3):
    ((tn, fp), (fn, tp)) =  mcf[i]
    tot +=tp/(tp+fp)
prec =  tot/3   
print(prec)

In [ ]:
precision_score(encoded_test_class, predicted_test_class, average='macro')

and $F_1$ score is

$$F_1 = \frac{1}{k}\sum_{i=0}^{k-1}\frac{2\cdot TP_i}{TP_i+FP_i +TP_i +FN_i}$$

In [ ]:
tot = 0
for i in range(3):
    ((tn, fp), (fn, tp)) =  mcf[i]
    tot +=2 * tp/(tp+fp+tp+fn)
f =  tot/3   
print(f)

In [ ]:
f1_score(encoded_test_class, predicted_test_class, average='macro')

### Weighted averaging 

And finally the weighted averaging is like macro averaging but we  weight the average by the support of each class _i.e._ the number of labels of each class. E.g. for precision

In [ ]:
tot = 0
den = 0
for i in range(3):
    tp, fp, fn, tn =  stat(true,pred, i)
    tot +=tp/(tp+fp) *(tp+fn)
    den += (tp+fn)
prec = tot/den
print(prec)

In [ ]:
precision_score(encoded_test_class, predicted_test_class, average='weighted')

#### Problem 

 Show that weighted averaging for recall gives same result as micro averaging.

As you may have guessed scikit-learn library has some convenient functions that calculate those metrics at once.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,  classification_report

In [ ]:
precision_recall_fscore_support(encoded_test_class, predicted_test_class, beta = 2, average = 'macro')

In [ ]:
print(classification_report(encoded_test_class, predicted_test_class))